WeatherPy

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress

# Import API key
from config import key_dict

# weather key
w_key = key_dict['owm']

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

Generate Cities List 

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []


# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        
#print the city count to confirm sufficient count
print(len(cities))

610


Perform API Calls

Perform a weather check on each city using a series of successive API calls.
Include a print log of each city as it's being processed (with the city number and city name).

In [7]:
#create lists 
city_list = []
cloud = []
country = []
date = []
humidity = []
lat = []
lng = []
max_temp = []
wind_speed = []
index_counter = 0
set_counter = 1

In [8]:
#perform API query and print log 
print("Beginning Data Retrieval ")
print("-------------------------------")

base_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
query_url = f"{base_url}appid={w_key}&units={units}&q="

#loop for cities
for index, city in enumerate(cities, start = 1):
    try:
        response = requests.get(query_url + city).json()
        city_list.append(response["name"])
        cloud.append(response["clouds"]["all"])
        country.append(response["sys"]["country"])
        date.append(response["dt"])
        humidity.append(response["main"]["humidity"])
        lat.append(response["coord"]["lat"])
        lng.append(response["coord"]["lon"])
        max_temp.append(response['main']['temp_max'])
        wind_speed.append(response["wind"]["speed"])
        if index_counter > 49:
            index_counter = 0
            set_counter = set_counter + 1
        
        else:
            index_counter = index_counter + 1
        
        print(f"Processing Record {index_counter} of Set {set_counter} : {city}")
        
    except(KeyError, IndexError):
        print("City not found. Skipping...")
        
print("-------------------------------")
print("Data Retrieval Complete")
print("-------------------------------")

Beginning Data Retrieval 
-------------------------------
Processing Record 1 of Set 1 : hobart
Processing Record 2 of Set 1 : provideniya
Processing Record 3 of Set 1 : port lincoln
Processing Record 4 of Set 1 : stornoway
Processing Record 5 of Set 1 : punta arenas
Processing Record 6 of Set 1 : kondopoga
Processing Record 7 of Set 1 : bredasdorp
Processing Record 8 of Set 1 : kijang
Processing Record 9 of Set 1 : namibe
City not found. Skipping...
Processing Record 10 of Set 1 : hermanus
City not found. Skipping...
Processing Record 11 of Set 1 : saint-augustin
Processing Record 12 of Set 1 : aden
Processing Record 13 of Set 1 : rikitea
City not found. Skipping...
Processing Record 14 of Set 1 : upernavik
Processing Record 15 of Set 1 : guiratinga
Processing Record 16 of Set 1 : vaini
Processing Record 17 of Set 1 : tuktoyaktuk
Processing Record 18 of Set 1 : new norfolk
City not found. Skipping...


KeyboardInterrupt: 

 Convert Raw Data to DataFrame

Export the city data into a .csv.Display the DataFrame

In [12]:
#Create a dataframe 
weather_info_df = pd.DataFrame({"City" : city_list,"Lat" : lat,"Lng" : lng,
    "Max Temp" : max_temp, "Humidity" : humidity,"Clouds" : cloud,
    "Wind Speed" : wind_speed, "Country" : country,"Date" : date    
})

weather_info_df.head()

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Hobart,-42.8794,147.3294,75.00,46,40,14.97,AU,1616298023
1,Provideniya,64.3833,-173.3000,7.29,76,100,9.69,RU,1616298035
2,Port Lincoln,-34.7333,135.8667,76.39,58,1,11.18,AU,1616298035
3,Stornoway,58.2093,-6.3865,44.01,87,90,6.91,GB,1616298036
4,Punta Arenas,-53.1500,-70.9167,50.00,66,40,29.93,CL,1616297761
